# <center>Итоговый проект первого года обучения Skillfactory Data Science

# <center>Определение уязвимых групп населения

## <center> 1. Описание данных
### Проектный анализ будет проводиться на предоставленных и загруженных из открытых источников интернета данных: ####

  ➔ **child_mortality_rural_1990_2021.xls** — число умерших на первом году жизни детей за год, по всем регионам,в сельской местности;

  ➔ **child_mortality_urban_1990_2021.xls** — число умерших на первом году жизни детей за год, по всем регионам,в городской местности;

  ➔ **morbidity_2005_2020_age_disease.xls** — заболеваемость на 100 тыс. человек населения, по возрастным группам и группам заболеваний;

  ➔ **poverty_percent_by_regions_1992_2020.csv** — процент людей, живущих за чертой бедности (с денежными доходами ниже величины прожиточного минимума), оценка за год по регионам;

  ➔ **welfare_expense_share_2015_2020** — расходы на социальную политику от общих расходов бюджета региона, % в год;

  ➔ **cash_real_income_wages_2015_2020** — среднедушевые и реальные денежные доходы населения, номинальная и реальная начисленная зарплата, по регионам;

  ➔ **poverty_socdem_2017-2020.xls** — распределение малоимущего населения по социально-демографическим группам (дети, трудящиеся, пенсионеры) за 2017–2020 гг., по регионам;

  ➔ **housing_2020.xlsx** — характеристика жилищных условий домохозяйств. Оценка домохозяйствами состояния занимаемого ими жилого помещения, обследование 2020 года;

  ➔ **population.xlsx** — численность населения по регионам и федеральным округам на 1 января каждого года за 1999–2022 гг;

  ➔ **gross_regional_product_1996_2020.xls** — валовой региональный продукт на душу населения, в рублях;

  ➔ **regional_production_2005_2016.csv** — объём регионального производства или работ/услуг, выполненных собственными силами, 2005–2016 гг. (в млн.руб., значение показателя за год, полный круг)*;

  ➔ **regional_production_2017-2020.csv** — объём регионального производства или работ/услуг, выполненных собственными силами, 2017–2020 гг. (в млн.руб., значение показателя за год, полный круг)*;

  ➔ **retail_turnover_per_capita_2000_2021.xls** — оборот розничной торговли на душу населения, в рублях;

  ➔ **newborn_2006_2022_monthly.csv** — рождаемость (за год) за период 2006-2022гг.*;

  ➔ **disabled_total_by_age_2017_2022.csv** — число людей с инвалидностью по регионам за период 2017-2022гг.

  ➔ **drug_alco.xlsx** — сведения о заболеваемости алкоголизмом и наркоманией, на 100 тыс. населения (2005–2018).

  ➔ Папка **crimes** — сведения о преступлениях, совершённых отдельными категориями лиц за 2016–2022гг., по месяцам, регионам, категориям лиц, категориям преступлений:
   1. В папке содержатся файлы о расследованных преступлениях (число преступлений);
   2. Файлы представлены в том виде, в котором их предоставляет Генпрокуратура.
    

## <center> 2. Подготовка данных

In [119]:
import numpy as np
import pandas as pd
import pickle
import regex as re

Функция для извлечения данных по федеральным субъектам и стандартизации их названий. 
- df - DataFrame;
- standard_names - словарь стандартных названий; 
- список names_to_drop позволяет сразу исключить некоторые регионы, которые могут быть ошибочно обработаны функцией как федеральные субъекты;
- region_col - колонка DataFrame с названиями регионов.

In [120]:
def federal_subj(
    df, standard_names, names_to_drop=None, region_col='region'):
    """Считывание данных по федеральным субъектам"""
       
    def get_standard_name(x):
        """Функция для .apply()
        Просмотр столбца регионов и вывод названий в стандартном виде"""
    
        for key, value in standard_names.items():
            x = re.sub('[\(].*?[\)]', '', x)  # удаляем всё, что в скобках ()
            if re.search(key, x.lower()):
                return value
        return None
    
    if names_to_drop is None:
        names_to_drop = []
    df = df[~df[region_col].isin(names_to_drop)].copy()
    df[region_col] = df[region_col].apply(get_standard_name)
    return (df.dropna(subset=region_col)
            .set_index(region_col).sort_index()) 

<>:10: SyntaxWarning: invalid escape sequence '\('
<>:10: SyntaxWarning: invalid escape sequence '\('
C:\Users\User\AppData\Local\Temp\ipykernel_3052\4105559176.py:10: SyntaxWarning: invalid escape sequence '\('
  x = re.sub('[\(].*?[\)]', '', x)  # удаляем всё, что в скобках ()


Зачитаем словарь `social_russia_data/standard_names.pkl`.

In [121]:
with open('social_russia_data/standard_names.pkl', 'rb') as f:
    standard_names = pickle.load(f)

### 1.1 Зачитаем данные о детской смертности (в сельской и городской местности)

In [122]:
def mortality_readout(file):
    "Считывание данных с excel-листа"
    df = pd.read_excel(file, header=2, skipfooter=13, usecols='A, D:AI')
    df.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
    names_to_drop = [
        '            Архангельская область (кроме Ненецкого автономного округа)',
        '            Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
    ]
    df = federal_subj(df, standard_names, names_to_drop)
    df.loc['Архангельская область без Ненецкого АО'] \
        = (df.loc['Архангельская область без Ненецкого АО']
           - df.loc['Ненецкий автономный округ'])
    df.loc['Тюменская область без округов'] \
        = (df.loc['Тюменская область без округов']
           - df.loc['Ханты-Мансийский автономный округ - Югра']
           - df.loc['Ямало-Ненецкий автономный округ'])
    df.columns = pd.to_datetime(df.columns, format='%Y').year
    return df


rural = mortality_readout(
    'social_russia_data/child_mortality_rural_1990_2021.xls')
urban = mortality_readout(
    'social_russia_data/child_mortality_urban_1990_2021.xls')
child_mortality = rural.add(urban)
child_mortality.head()

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,522.0,567.0,529.0,560.0,497.0,490.0,405.0,375.0,340.0,391.0,...,330.0,290.0,275.0,222.0,211.0,185.0,176.0,114.0,97.0,88.0
Амурская область,343.0,293.0,280.0,244.0,242.0,250.0,245.0,268.0,246.0,270.0,...,161.0,119.0,101.0,83.0,53.0,48.0,46.0,43.0,42.0,36.0
Архангельская область без Ненецкого АО,288.0,274.0,245.0,237.0,224.0,206.0,194.0,157.0,167.0,168.0,...,108.0,113.0,97.0,84.0,79.0,72.0,53.0,58.0,29.0,32.0
Астраханская область,250.0,235.0,188.0,216.0,202.0,194.0,192.0,188.0,221.0,154.0,...,142.0,130.0,148.0,119.0,82.0,74.0,71.0,72.0,64.0,46.0
Белгородская область,312.0,290.0,240.0,244.0,233.0,203.0,174.0,167.0,160.0,161.0,...,127.0,116.0,103.0,110.0,108.0,77.0,73.0,39.0,47.0,54.0


### 1.2 Зачитаем данные по заболеваемости

Для обеспечения единой методики расчета по Архангельской и Тюменской областям берем данные с 2014 г. (данные рассчитывались в перерасчете на 100 тыс. человек, при этом статистика по областям без автономных округов до 2014 г. не приводится). Данные позже 2016 г. отсутствуют. Выделяем данные по диагнозам, которые могут быть связаны с социальным неблагополучием.

In [123]:
morbidity = pd.read_excel(
        'social_russia_data/morbidity_2005_2020_age_disease.xls',
        header=2, usecols='A:C, M:O')
morbidity.rename(
    {'Unnamed: 0': 'region', 'Unnamed: 1': 'type', 'Unnamed: 2': 'age'}, 
    axis=1, inplace=True)
morbidity = morbidity.dropna(subset=['2014', '2015', '2016'], how='all')
morbidity = federal_subj(morbidity, standard_names)

# Общая заболеваемость
mask = (
    (morbidity['type'] == 'Все заболевания')
    & (morbidity['age'] ==  'Всего')   
)
general_morbidity = morbidity[mask].copy()
general_morbidity.drop(columns=['type', 'age', '2014'], inplace=True)
general_morbidity.columns = pd.to_datetime(
    general_morbidity.columns).year

# Врожденные пороки развития
mask = (
    (morbidity['type'] == 'Врожденные аномалии (пороки развития), деформации и хромосомные нарушения')
    & (morbidity['age'] ==  'Всего')   
)
congenital_malformation = morbidity[mask].copy()
congenital_malformation.drop(columns=['type', 'age'], inplace=True)
congenital_malformation.columns = pd.to_datetime(
    congenital_malformation.columns).year

# Психические расстройства и расстройства поведения
mask = (
    (morbidity['type'] == 'Психические расстройства и расстройства поведения')
    & (morbidity['age'] ==  'Всего')   
)
psychosis = morbidity[mask].copy()
psychosis.drop(columns=['type', 'age'], inplace=True)
psychosis.columns = pd.to_datetime(
    psychosis.columns).year

# Подростковая беременность (до 14 лет)
mask = (
    (morbidity['type'] == 'Беременность, роды и послеродовой период')
    & (morbidity['age'] ==  '0-14 лет')   
)
teenage_pregnancy = morbidity[mask].copy()
teenage_pregnancy.drop(columns=['type', 'age'], inplace=True)
teenage_pregnancy.columns = pd.to_datetime(
    teenage_pregnancy.columns).year

display('Общая заболеваемость', general_morbidity.head(3))
display('Врожденные пороки развития', congenital_malformation.head(3))
display('Психические расстройства', psychosis.head(3))
display('Подростковая беременность', teenage_pregnancy.head(3))

'Общая заболеваемость'

,2015,2016
region,,
Алтайский край,109652.6,112764.7
Амурская область,83919.0,83412.0
Архангельская область без Ненецкого АО,101587.4,99842.1


'Врожденные пороки развития'

,2014,2015,2016
region,,,
Алтайский край,176.1,196.7,200.1
Амурская область,391.2,335.2,370.1
Архангельская область без Ненецкого АО,380.8,423.9,458.9


'Психические расстройства'

,2014,2015,2016
region,,,
Алтайский край,1454.0,1467.7,1272.1
Амурская область,512.4,485.3,446.7
Архангельская область без Ненецкого АО,646.0,693.5,685.3


'Подростковая беременность'

,2014,2015,2016
region,,,
Алтайский край,3.4,8.3,3.3
Амурская область,22.4,13.3,4.4
Архангельская область без Ненецкого АО,10.5,6.9,13.8


### 1.3 Зачитаем данные о численности населения за чертой бедности

In [124]:
poverty_percent = pd.read_csv(
    'social_russia_data/poverty_percent_by_regions_1992_2020.csv')
poverty_percent = pd.pivot_table(
    data=poverty_percent, values='poverty_percent', 
    columns='year', index='region').reset_index()
names_to_drop = [
    '        Архангельская область',
    '        Тюменская область'
]
poverty_percent = federal_subj(poverty_percent, standard_names, names_to_drop)
poverty_percent.columns = pd.to_datetime(
    poverty_percent.columns, format='%Y').year

display(poverty_percent.head())

year,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,NaN,NaN,NaN,33.7,46.8,45.7,52.9,53.8,53.9,47.3,...,22.6,20.6,17.6,17.1,18.0,17.8,17.5,17.4,17.6,17.5
Амурская область,NaN,NaN,NaN,36.1,28.2,26.3,31.2,38.0,47.7,45.3,...,20.4,16.0,16.2,14.8,15.2,17.0,16.7,15.6,15.7,15.2
Архангельская область без Ненецкого АО,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,13.0,14.5,14.5,16.2,14.9,13.9,12.5,12.7,12.3
Астраханская область,NaN,NaN,NaN,32.3,30.7,25.0,30.4,36.0,33.4,31.1,...,14.2,12.5,11.8,12.0,14.2,16.1,15.5,15.1,15.5,15.6
Белгородская область,NaN,NaN,NaN,19.9,17.6,18.9,20.1,24.2,33.6,29.1,...,8.6,6.5,7.6,7.7,8.5,8.1,7.8,7.5,7.8,7.2


### 1.4 Зачитаем расходы на социальную политику, % от общих расходов бюджета региона за период 2015-2020гг в год

In [125]:
welfare_expense_share = pd.read_excel('social_russia_data/welfare_expense_share_2015_2020.xlsx', sheet_name=0)
welfare_expense_share = federal_subj(welfare_expense_share, standard_names)
display(welfare_expense_share.head())

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,18.8,20.4,30.9,29.8,28.3,29.7
Амурская область,19.2,28.9,26.3,24.1,22.7,21.7
Архангельская область без Ненецкого АО,15.4,16.3,24.8,24.2,22.4,21.4
Астраханская область,18.8,20.5,27.7,26.4,25.8,31.4
Белгородская область,11.3,11.8,16.4,15.6,13.8,16.1


### 1.5 Зачитаем данные о доходах населения за 2015 - 2020 годы

In [126]:
def read_finances(file, sheet):
    "Считывание финансовых данных с excel-листа"
    df = pd.read_excel(file, sheet_name=sheet)
    names_to_drop = ['Архангельская область', 'Тюменская область']
    df = federal_subj(df, standard_names, names_to_drop)
    df.columns = pd.to_datetime((df.columns).astype(int), format='%Y').year
    return df

file = './social_russia_data/cash_real_income_wages_2015_2020.xlsx'
nominal_incomes = read_finances(file, 0)
real_incomes = read_finances(file, 1)
formal_wage = read_finances(file, 2)
real_pay = read_finances(file, 3)

display('Номинальный доход на душу населения', nominal_incomes.head(3))
display('Реальный доход, %', real_incomes.head())
display('Номинальная зарплата', formal_wage.head())
display('Реальная зарпалата, %', real_pay.head())

'Номинальный доход на душу населения'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,20860.0,21256.0,22139.0,22829.0,23937.0,23864.0
Амурская область,28240.0,27976.0,29213.0,30937.0,33304.0,35499.0
Архангельская область без Ненецкого АО,29716.0,29837.0,30707.0,32054.0,33874.0,34852.0


'Реальный доход, %'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,99.1,94.7,100.0,99.7,99.6,95.5
Амурская область,96.1,92.1,101.1,102.4,101.7,100.3
Архангельская область без Ненецкого АО,95.1,93.0,98.7,101.7,100.2,98.4
Астраханская область,94.0,90.1,97.1,100.6,100.7,97.1
Белгородская область,99.3,100.8,99.1,98.7,100.6,98.1


'Номинальная зарплата'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,20090.0,21202.0,22743.0,25519.0,27962.0,30072.0
Амурская область,32902.0,33837.0,37368.0,42315.0,47234.0,52430.0
Архангельская область без Ненецкого АО,35592.0,38118.0,40352.0,45427.0,49435.0,52779.0
Астраханская область,25499.0,27493.0,29599.0,33630.0,36093.0,38885.0
Белгородская область,25456.0,27091.0,29066.0,31852.0,34615.0,37442.0


'Реальная зарпалата, %'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,90,98.4,103.6,109.3,104.9,103.8
Амурская область,88,96.0,107.4,110.1,106.0,105.2
Архангельская область без Ненецкого АО,92.4,100.0,102.5,110.7,104.0,102.9
Астраханская область,89.9,101.4,104.5,110.7,103.0,104.5
Белгородская область,93.2,100.8,104.5,106.8,104.0,104.8


### 1.6 Зачитаем данные о жилищных условиях за 2020 г

Выделяем: 
- среднюю жилую площадь на одного человека;
- процент домохозяйств, указавших на плохое состояние жилого помещения;
- процент домохозяйств, не имеющих средств для улучшения жилищных условий (процент встающих в очередь на жилье или рассчитывающих на переселение в связи со сносом дома)

In [127]:
def housing_readout(file, sheet, cols, skipfooter=0):
    "Считывание данных с excel-листа"
    df = pd.read_excel(
        file, sheet_name=sheet, usecols=cols, skipfooter=skipfooter)
    df.rename({'Регион': 'region'}, axis=1, inplace=True)
    
    # Ошибки в написании названий (латиница?)
    df.replace({'Bладимирская область': 'Владимирская'}, inplace=True)
    df.replace({'Bолгоградская область': 'Волгоградская'}, inplace=True)
    df.replace({'Bологодская область': 'Вологодская'}, inplace=True)
    df.replace({'Bоронежская область': 'Воронежская'}, inplace=True)
    df.replace('...', 0, inplace=True)  # слабая статистика -> 0
    df = federal_subj(df, standard_names)
    return df


file = 'social_russia_data/housing_2020.xlsx'
living_area = housing_readout(file, 0, 'A, H')
living_area.columns.values[0] = 'living area'
intents = housing_readout(file, 1, 'A, C, E, H, I', 2)
intents.columns.values[0] = 'to improve'
intents.columns.values[1] = 'bad conditions'
intents.columns.values[2] = 'queue'
intents.columns.values[3] = 'demolition'
intents['no funds'] = ((intents['to improve']/100 
    * (intents['queue']/100 + intents['demolition']/100)) * 100)
housing = pd.concat(
    [living_area, intents[['bad conditions', 'no funds']]], axis=1)
display(housing.head())

C:\Users\User\AppData\Local\Temp\ipykernel_3052\88210353.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('...', 0, inplace=True)  # слабая статистика -> 0


,living area,bad conditions,no funds
region,,,
Алтайский край,18.0,0.3,0.8642
Амурская область,17.3,1.3,1.0731
Архангельская область без Ненецкого АО,16.9,1.4,1.2998
Астраханская область,19.2,0.8,2.9028
Белгородская область,17.8,0.7,0.3840


### 1.7 Зачитаем данные о численности населения, кол-во человек

In [128]:
population = pd.read_excel(
        'social_russia_data/population.xlsx', header=1, usecols='A, C:Z')
population.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)

for i, val in enumerate(population['region']):
    if val == 'Раздел 1. Муниципальные образования субъектов Российской Федерации':
        population.loc[i, 'region'] = population.loc[i-1, 'region']
population.dropna(subset=population.columns[1:], how='all', inplace=True)

names_to_drop = [
        'Архангельская область (кроме Ненецкого автономного округа)',
        'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
        'Муниципальные образования Алтайского края',
        'Алтайский муниципальный район',
        'Алтайский сельсовет',
        'Большекалтайский сельсовет',
        'Населенные пункты, входящие в состав муниципальных образований Алтайского края',
        'Кировский сельсовет'
]
population = federal_subj(population, standard_names, names_to_drop)
population.loc['Архангельская область без Ненецкого АО'] \
    = (population.loc['Архангельская область без Ненецкого АО']
      - population.loc['Ненецкий автономный округ'])
population.loc['Тюменская область без округов'] \
    = (population.loc['Тюменская область без округов']
       - population.loc['Ханты-Мансийский автономный округ - Югра']
       - population.loc['Ямало-Ненецкий автономный округ'])

newcols = []
for col in population.columns:
    newcols.append(re.search(r'\d+', col).group(0))
population.columns = pd.to_datetime(newcols).year

display(population.head())

,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,2662738.0,2651628.0,2641079.0,2621050.0,2602595.0,2571987.0,2539430.0,2503510.0,2473024.0,2453455.0,...,2398751.0,2390638.0,2384812.0,2376774.0,2365680.0,2350080.0,2332813.0,2317153.0,2296353.0,2268179.0
Амурская область,949526.0,935607.0,923055.0,911381.0,901044.0,887781.0,874018.0,861056.0,850502.0,844290.0,...,816910.0,811274.0,809873.0,805689.0,801752.0,798424.0,793194.0,790044.0,781846.0,772525.0
Архангельская область без Ненецкого АО,1372631.0,1349160.0,1328187.0,1309552.0,1290956.0,1273668.0,1257312.0,1239924.0,1224813.0,1215264.0,...,1159506.0,1148760.0,1139950.0,1130240.0,1121813.0,1111031.0,1100290.0,1092424.0,1082662.0,1069782.0
Астраханская область,1016372.0,1012385.0,1009281.0,1005510.0,1004780.0,1006073.0,1006467.0,1002517.0,1001300.0,1005897.0,...,1013840.0,1016516.0,1021287.0,1018626.0,1018866.0,1017514.0,1014065.0,1005782.0,997778.0,989430.0
Белгородская область,1494868.0,1501699.0,1506976.0,1508137.0,1511899.0,1513860.0,1511662.0,1511715.0,1514153.0,1520102.0,...,1540985.0,1544108.0,1547936.0,1550137.0,1552865.0,1549876.0,1547418.0,1549151.0,1541259.0,1531917.0


### 1.8 Зачитаем данные о ВВП регионов

Считываем данные с 2011 года - обеспечивается единая методика расчета по Архангельской и Тюменской областям.

In [129]:
grp = pd.read_excel('social_russia_data/gross_regional_product_1996_2020.xls', 
                    header=2, skipfooter=1, usecols='A, Q:Z')
grp.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
names_to_drop = [
    '        Архангельская область',
    '        Тюменская область']
grp = federal_subj(grp, standard_names, names_to_drop)
grp.columns = pd.to_datetime(grp.columns).year
display(grp.head())

,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
region,,,,,,,,,,
Алтайский край,137677.2,153556.7,173763.5,186798.6,204933.1,224525.8,231268.4,247599.3,271319.7,291156.9
Амурская область,273175.8,280023.9,258817.0,286282.6,343385.7,370192.4,373935.1,419905.2,521060.1,571362.1
Архангельская область без Ненецкого АО,232540.7,270662.9,283264.5,310817.4,352837.9,400764.6,441961.6,493205.1,509917.0,514200.4
Астраханская область,170504.7,206677.1,269821.7,290822.2,315996.9,361704.8,434701.5,570206.4,596388.2,526950.9
Белгородская область,331010.0,354570.6,368874.8,400820.8,447619.7,501467.8,539720.5,588641.5,617426.5,646569.0


### 1.9 Зачитаем данные об объёме регионального производства или работ/услуг.

In [130]:
reg_prod_new = pd.read_csv(
    'social_russia_data/regional_production_2017_2020.csv')
names_to_drop = [
    '        Архангельская область', 
    '        Тюменская область']
reg_prod_new = federal_subj(reg_prod_new, standard_names, names_to_drop) 
reg_prod_new.columns.values[-4:] = pd.to_datetime(reg_prod_new.columns[-4:]).year

reg_prod_old = pd.read_csv(
    'social_russia_data/regional_production_2005_2016.csv')
reg_prod_old = federal_subj(reg_prod_old, standard_names, names_to_drop)
reg_prod_old.columns.values[-12:] = pd.to_datetime(reg_prod_old.columns[-12:]).year

reg_prod = pd.concat([reg_prod_old, reg_prod_new])
reg_prod_total = (reg_prod
                  .groupby('region')[[2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]].sum())
reg_prod_total.columns = pd.to_datetime(
    reg_prod_total.columns, format='%Y').year

display('Региональное производство, по видам деятельности', reg_prod.head())
display('Региональное производство, общий объем', reg_prod_total.head())

'Региональное производство, по видам деятельности'

,production_field,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
region,,,,,,,,,,,,,,,,,
Алтайский край,РАЗДЕЛ E ПРОИЗВОДСТВО И РАСПРЕДЕЛЕНИЕ ЭЛЕКТРОЭ...,21776598.5,23693231.2,19258417.8,2.389591e+07,26748131.7,3.179362e+07,3.199120e+07,3.439760e+07,3.772056e+07,4.084313e+07,4.435540e+07,4.504698e+07,NaN,NaN,NaN,NaN
Алтайский край,РАЗДЕЛ С ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,461065.8,968509.5,2238427.0,3.095174e+06,3383227.4,4.402534e+06,6.040719e+06,7.208823e+06,4.068447e+06,3.637479e+06,5.519930e+06,4.852175e+06,NaN,NaN,NaN,NaN
Алтайский край,РАЗДЕЛ D ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА,70381505.7,91823076.0,108789152.1,1.546641e+08,118002216.6,1.687516e+08,1.892789e+08,1.989759e+08,2.056299e+08,2.098569e+08,2.716510e+08,2.658828e+08,NaN,NaN,NaN,NaN
Амурская область,РАЗДЕЛ С ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,7941909.4,9576979.1,10649342.6,1.513225e+07,23995368.4,2.659908e+07,4.997345e+07,5.347346e+07,4.974698e+07,5.569035e+07,7.217962e+07,6.437910e+07,NaN,NaN,NaN,NaN
Амурская область,РАЗДЕЛ D ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА,8639523.2,9620159.5,11343578.7,1.403151e+07,15097496.3,1.856792e+07,2.082908e+07,2.202458e+07,2.195232e+07,2.405030e+07,3.390215e+07,2.789874e+07,NaN,NaN,NaN,NaN


'Региональное производство, общий объем'

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
region,,,,,,,,,,,,,,,
Алтайский край,116484816.7,1.302860e+08,1.816552e+08,148133575.7,2.049478e+08,2.273109e+08,2.405824e+08,2.474189e+08,2.543375e+08,3.215263e+08,3.157819e+08,7.359764e+08,7.711927e+08,8.383179e+08,8.524434e+08
Амурская область,35861289.8,3.875065e+07,4.679807e+07,60866961.2,7.210232e+07,9.669124e+07,1.024127e+08,9.924642e+07,1.093931e+08,1.386028e+08,1.259368e+08,2.688441e+08,2.687608e+08,3.400267e+08,4.045765e+08
Архангельская область без Ненецкого АО,0.0,0.000000e+00,0.000000e+00,0.0,1.264137e+08,1.330191e+08,1.503460e+08,2.980072e+08,1.878591e+08,2.115037e+08,4.061885e+08,7.090098e+08,6.064711e+08,5.335849e+08,6.641070e+08
Астраханская область,45566202.3,5.194887e+07,8.469228e+07,62969236.5,7.668357e+07,9.572381e+07,1.253555e+08,1.623128e+08,1.730122e+08,2.116270e+08,2.277982e+08,6.265697e+08,8.837665e+08,9.015147e+08,7.245978e+08
Белгородская область,194782787.5,2.648646e+08,3.501343e+08,295771628.7,4.157647e+08,5.043765e+08,5.289788e+08,5.293518e+08,5.818284e+08,6.728485e+08,7.210363e+08,1.566651e+09,1.790868e+09,1.878073e+09,1.948294e+09


### 1.10 Загрузим данные оборота розничной торговли на душу населения

In [131]:
retail = pd.read_excel(
    'social_russia_data/retail_turnover_per_capita_2000_2021.xls', 
    header=2, usecols='A, P:X')
retail.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
retail.drop(0, inplace=True)
names_to_drop = [
        '        Архангельская область',
        '            Архангельская область (без АО)',
        '        Тюменская область',
        '            Тюменская область (без АО)'
]
retail = federal_subj(retail, standard_names, names_to_drop)
retail.columns = pd.to_datetime(retail.columns).year

display(retail.head())

,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,
Алтайский край,118096,128376,134925,137844,143873,150444,159514,153605,174321
Амурская область,145301,163781,182491,191523,202038,214688,231113,245233,276635
Архангельская область без Ненецкого АО,154177,176420,194345,202977,217332,229922,240155,250033,280050
Астраханская область,147954,162393,170883,164241,163829,170710,179153,174527,196096
Белгородская область,144992,164079,178097,192504,203298,217059,230704,237076,259151


### 1.11 Загрузим данные о рождаемости

In [132]:
newborn = pd.read_csv(
    'social_russia_data/newborn_2006_2022_monthly.csv', sep=';', decimal=',')
newborn.drop(columns='Unnamed: 198',inplace=True)
newborn.rename({'Region': 'region'}, axis=1, inplace=True)
names_to_drop = [
    'Архангельская область (кроме Ненецкого автономного округа)',
    'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
    'Московская обл. в старых границах',
    'Москва в старых границах'
]
newborn = federal_subj(newborn, standard_names, names_to_drop=names_to_drop)
newborn.loc['Архангельская область без Ненецкого АО'] \
    = (newborn.loc['Архангельская область без Ненецкого АО']
      - newborn.loc['Ненецкий автономный округ'])
newborn.loc['Тюменская область без округов'] \
    = (newborn.loc['Тюменская область без округов']
       - newborn.loc['Ханты-Мансийский автономный округ - Югра']
       - newborn.loc['Ямало-Ненецкий автономный округ'])
    
# Суммируем данные за год
for year in range(2006, 2023):
    cols = []
    for col in newborn.columns:
        if str(year) in str(col):
            cols.append(col)
    newborn[year] = newborn[cols].sum(axis=1)
    newborn.drop(columns=cols, inplace=True)
newborn.drop(columns=2022, inplace=True)  # Неполный год
newborn = newborn.round(0).astype(int)
newborn.columns = pd.to_datetime(newborn.columns, format='%Y').year

display(newborn.head())

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,
Алтайский край,25513,28209,30711,31038,30863,30739,32695,32461,31429,30065,28867,25525,23491,21117,19976,18992
Амурская область,10405,10935,11233,11384,11473,11188,11733,11483,11159,10781,10377,9430,8889,7919,7853,7343
Архангельская область без Ненецкого АО,13396,14438,14577,14776,14806,14253,14730,14584,14331,13908,13313,11714,10671,9662,8906,8487
Астраханская область,12357,13370,14200,14350,14355,14378,15304,15042,15216,14875,14270,12321,11800,11005,10911,10709
Белгородская область,14269,15666,16810,16911,16669,16894,17913,17883,17822,17864,17261,15113,14318,13198,12352,12223


### 1.12 Загрузим данные о числе людей с инвалидностью по регионам

Единица измерения - количество человек. Таблица содержит данные по месяцам и для разных возрастов, берем данные на 1 января каждого года по непенсионным (до 60 лет) возрастам: количество инвалидов пожилого возраста может зависеть, в том, числе, от естественного старения. Инвалидность среди трудоспособного населения дает более точную оценку социального неблагополучия региона.  

In [133]:
disabled = pd.read_csv(
    'social_russia_data/disabled_total_by_age_2017_2022.csv')
disabled = federal_subj(disabled, standard_names)

disabled['date'] = pd.to_datetime(disabled['date'])
disabled['dayofyear'] = disabled['date'].dt.dayofyear
disabled = disabled[disabled['dayofyear'] == 1]
disabled['young'] = disabled[['18_30', '31_40', '41_50', '51_60']].sum(axis=1)
disabled = pd.pivot_table(
    data=disabled, values='young', index='region', columns='date')
disabled.columns = (disabled.columns).year
display(disabled.head())

date,2017,2018,2019,2020,2021,2022
region,,,,,,
Алтайский край,66770.0,64605.0,62840.0,61378.0,59810.0,59006.0
Амурская область,27892.0,26720.0,25976.0,25069.0,24149.0,23356.0
Архангельская область без Ненецкого АО,29228.0,28262.0,27615.0,26984.0,26498.0,26009.0
Астраханская область,21018.0,20394.0,19822.0,19534.0,19391.0,19313.0
Белгородская область,70741.0,66742.0,63964.0,60716.0,58208.0,55509.0


### 1.13 Загрузим сведения о преступлениях, совершённых отдельными категориями лиц за 2020 г.

Берем 2020 год, так как он содержит наиболее полные сведения по предыдущим таблицам.

Будем использовать данные о преступлениях, совершенных несовершеннолетними или при их соучастии и ранее совершавшими преступления. Считаем, что это две независимые группы, поэтому будем использовать их сумму.

В итоговую таблицу возьмем данные по всем преступлениям, а также отдельно по видам тяжести.

In [134]:
file = 'social_russia_data/crimes/4-EGS_Razdel_4_122020.xls'

# Данные по всем преступлениям
crimes_all = pd.read_excel(
    file, header=11, sheet_name=0, usecols='B, C, D')
crimes_all['all crimes'] = crimes_all[1] + crimes_all[2]
crimes_all.drop([1, 2], axis=1, inplace=True)
crimes_all.rename({'Unnamed: 1': 'region'}, axis=1, inplace=True)
crimes_all = federal_subj(crimes_all, standard_names)

# Данные по особо тяжким преступлениям
crimes_very_hard = pd.read_excel(
    file, header=11, sheet_name=1, usecols='B, C, D')
crimes_very_hard['very hard crimes'] = crimes_very_hard[1] + crimes_very_hard[2]
crimes_very_hard.drop([1, 2], axis=1, inplace=True)
crimes_very_hard.rename({'Unnamed: 1': 'region'}, axis=1, inplace=True)
crimes_very_hard = federal_subj(crimes_very_hard, standard_names)

# Данные по тяжким преступлениям
crimes_hard = pd.read_excel(
    file, header=11, sheet_name=2, usecols='B, C, D')
crimes_hard['hard crimes'] = crimes_hard[1] + crimes_hard[2]
crimes_hard.drop([1, 2], axis=1, inplace=True)
crimes_hard.rename({'Unnamed: 1': 'region'}, axis=1, inplace=True)
crimes_hard = federal_subj(crimes_hard, standard_names)

# Данные по преступлениям средней тяжести
crimes_middle_hard = pd.read_excel(
    file, header=11, sheet_name=3, usecols='B, C, D')
crimes_middle_hard['middle hard crimes'] = crimes_middle_hard[1] + crimes_middle_hard[2]
crimes_middle_hard.drop([1, 2], axis=1, inplace=True)
crimes_middle_hard.rename({'Unnamed: 1': 'region'}, axis=1, inplace=True)
crimes_middle_hard = federal_subj(crimes_middle_hard, standard_names)

# Данные по преступлениям небольшой тяжести
crimes_little_hard = pd.read_excel(
    file, header=11, sheet_name=4, usecols='B, C, D')
crimes_little_hard['little hard crimes'] = crimes_little_hard[1] + crimes_little_hard[2]
crimes_little_hard.drop([1, 2], axis=1, inplace=True)
crimes_little_hard.rename({'Unnamed: 1': 'region'}, axis=1, inplace=True)
crimes_little_hard = federal_subj(crimes_little_hard, standard_names)

crimes = pd.concat([crimes_all['all crimes'], crimes_very_hard['very hard crimes'], crimes_hard['hard crimes'], crimes_middle_hard['middle hard crimes'], crimes_little_hard['little hard crimes']], axis=1)
display(crimes.head())

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,all crimes,very hard crimes,hard crimes,middle hard crimes,little hard crimes
region,,,,,
Алтайский край,16287,454,2435,3882,9516
Амурская область,6432,224,1359,1635,3214
Архангельская область без Ненецкого АО,7120,298,1160,1873,3789
Астраханская область,5738,283,779,1382,3294
Белгородская область,4818,133,650,1119,2916


### 1.14 Загрузим сведения о заболеваемости алкоголизмом и наркоманией.

Единица измерения - количество акогольных/наркотических психозов на 100 тыс. населения.

In [135]:
def alco_readout(file, sheet):
    "Считывание данных с excel-листа"
    df = pd.read_excel(file, sheet_name=sheet)
    df = federal_subj(df, standard_names)
    df.columns = pd.to_datetime((df.columns).astype(int), format='%Y').year
    return df


file = 'social_russia_data/drug_alco.xlsx'
alco = alco_readout(file, 0)
alco1718 = alco_readout(file, 1)
drug = alco_readout(file, 2)
drug1718 = alco_readout(file, 3)
alco = pd.concat([alco, alco1718], axis=1).sort_index()
drug = pd.concat([drug, drug1718], axis=1).sort_index()

display('Алкоголизм', alco.head(3))
display('Наркомания', drug.head(3))

'Алкоголизм'

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
region,,,,,,,,,,,,,,
Алтайский край,227.6,202.4,173.9,199.8,177.9,164.0,157.6,138.9,105.0,111.9,115.7,100.1,85.9,88.1
Амурская область,174.3,159.0,120.9,113.3,115.5,140.5,124.7,104.8,100.0,96.7,97.2,92.3,91.4,113.3
Архангельская область без Ненецкого АО,120.1,115.2,102.5,89.8,96.5,99.5,84.4,73.2,77.6,NaN,NaN,NaN,55.3,64.7


'Наркомания'

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
region,,,,,,,,,,,,,,
Алтайский край,29.8,35.2,35.2,37.7,27.3,24.5,26.4,33.4,22.9,21.6,20.7,17.4,17.4,17.4
Амурская область,21.3,27.2,27.2,28.4,26.2,37.1,33.9,24.2,22.5,22.2,19.1,18.9,18.7,24.4
Архангельская область без Ненецкого АО,1.5,1.4,1.4,2.6,3.4,4.6,6.4,4.0,4.0,NaN,NaN,NaN,5.4,9.2


### 1.15 Сформируем итоговый датасет для обучение модели.

Составим таблицу, содержащую все загруженные датасеты.

| Название датасета | Описание | Единица измерения | Период |
|-|-|-|-|
| child_mortality | Число умерших на первом году жизни детей за год | чел. | 1990-2021 |
| general_morbidity, congenital_malformation, psychosis, teenage_pregnancy | Общая заболеваемость, врожденные пороки развития, психические расстройства, подростковая беременность | кол-во. / 100 тыс. населения | 2015-2016 |
| poverty_percent | Процент населения за чертой бедности | %  | 1995-2020 |
| welfare_expense_share | Расходы на социальную политику от общих расходов бюджета региона | % | 2015 - 2020 |
| nominal_incomes | Средний денежный доход (в месяц) | руб. / чел. | 2015-2020 |
| real_incomes | Реальные денежные доходы (покупательная способность: с учетом роста цен) | процент к предыдущему году | 2015-2020 |
| formal_wage |Среднемесячная номинальная начисленная заработная плата | руб. / чел. | 2015-2020 |
| real_pay | Реальная начисленная заработная плата (покупательная способность: с учетом роста цен) | процент к предыдущему году | 2015-2020 |
| housing | Жилищные условия: средняя жилая площадь, плохое состояние жилья, отсутствие средств | кв.м., %, % | 2020 |
| population | Население | чел. | 1999-2022 |
| grp | Валовый региональный продукт на душу населения | руб. / чел. | 2011-2020 |
| reg_prod, reg_prod_total | Региональное производство (по видам деятельности или работ/услуг, общее) | тыс. руб. | 2005-2020 |
| retail | Розничная торговля | руб. / чел. | 2013-2021 |
| newborn | Рождаемость | чел. | 2006-2021 |
| disabled | Инвалидность среди населения трудоспособного возраста (18-60 лет) | чел. | 2017-2022 |
| crimes | Данные о преступлениях (общее количество, особо тяжкиеб тяжкие, средней тяжести и небольшой тяжести) | Кол-во | 2020 |
| alco| Сведения о заболеваемости алкоголизмом | кол-во. / 100 тыс. населения | 2005-2018 |
| drug | Сведения о заболеваемости наркоманией | кол-во. / 100 тыс. населения | 2005-2018 |

Сформируем общий датасет. За основу возьмем данные за 2020 год (представлен наиболее полно). Данные по общей заболеваемости возьмем за 2016 год, данные по заболеваемости алкоголизмом и наркоманией - за 2018 год.

Признак детской смертности возьмем по отношению к числу родившихся, в %.

Региональное производство возьмем по отношению к численности населения, в тыс. руб/чел.

Рождаемость посчитаем по отношению к численности населения, в %.

Инвалидность также посчитаем по отношению к численности населения, в %.

In [148]:
data = pd.concat([(child_mortality[[2020]] / newborn[[2020]] * 100).rename({2020:'child_mortality_percent'}, axis=1),
                  general_morbidity[[2016]].rename({2016:'general_morbidity'}, axis=1),
                  congenital_malformation[[2016]].rename({2016:'congenital_malformation'}, axis=1),
                  psychosis[[2016]].rename({2016:'psychosis'}, axis=1),
                  teenage_pregnancy[[2016]].rename({2016:'teenage_pregnancy'}, axis=1),
                  child_mortality[[2020]].rename({2020:'child_mortality'}, axis=1),
                  poverty_percent[[2020]].rename({2020:'poverty_percent'}, axis=1),
                  welfare_expense_share[[2020]].rename({2020:'welfare_expense_share'}, axis=1),
                  nominal_incomes[[2020]].rename({2020:'nominal_incomes'}, axis=1),
                  real_incomes[[2020]].rename({2020:'real_incomes'}, axis=1),
                  formal_wage[[2020]].rename({2020:'formal_wage'}, axis=1),
                  real_pay[[2020]].rename({2020:'real_pay'}, axis=1),
                  housing[['living area', 'bad conditions', 'no funds']],
                  population[[2020]].rename({2020:'population'}, axis=1),
                  grp[[2020]].rename({2020:'grp'}, axis=1),
                  reg_prod_total[[2020]].rename({2020:'reg_prod_total'}, axis=1),
                  (reg_prod_total[[2020]] / population[[2020]]).rename({2020:'reg_prod_total'}, axis=1),
                  retail[[2020]].rename({2020:'retail'}, axis=1),
                  (newborn[[2020]] / population[[2020]]  * 100).rename({2020:'newborn_per_population'}, axis=1),
                  (disabled[[2020]]/population[[2020]] * 100).rename({2020:'disabled_percent'}, axis=1),
                  crimes[['all crimes', 'very hard crimes', 'hard crimes', 'middle hard crimes', 'little hard crimes']],
                  alco[[2018]].rename({2018:'alco'}, axis=1),
                  drug[[2018]].rename({2018:'drug'}, axis=1)			
                  ],                  
                 axis=1)
display(data.head())

data.to_csv('social_russia_data/data_regions_2020.csv')

,child_mortality_percent,general_morbidity,congenital_malformation,psychosis,teenage_pregnancy,child_mortality,poverty_percent,welfare_expense_share,nominal_incomes,real_incomes,...,retail,newborn_per_population,disabled_percent,all crimes,very hard crimes,hard crimes,middle hard crimes,little hard crimes,alco,drug
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,0.485583,112764.7,200.1,1272.1,3.3,97.0,17.5,29.7,23864.0,95.5,...,153605,0.862092,2.648854,16287,454,2435,3882,9516,88.1,17.4
Амурская область,0.534827,83412.0,370.1,446.7,4.4,42.0,15.2,21.7,35499.0,100.3,...,245233,0.993995,3.173114,6432,224,1359,1635,3214,113.3,24.4
Архангельская область без Ненецкого АО,0.325623,99842.1,458.9,685.3,13.8,29.0,12.3,21.4,34852.0,98.4,...,250033,0.815251,2.470103,7120,298,1160,1873,3789,64.7,9.2
Астраханская область,0.586564,64679.9,415.1,624.5,0.0,64.0,15.6,31.4,25199.0,97.1,...,174527,1.084828,1.942170,5738,283,779,1382,3294,22.2,0.2
Белгородская область,0.380505,74637.4,136.2,388.2,0.0,47.0,7.2,16.1,32841.0,98.1,...,237076,0.797340,3.919308,4818,133,650,1119,2916,41.3,3.2


### Вывод

Была составлена общая таблица данных по регионам. За основу был взяты данные в основном за 2020 год. Итоговая таблица сохранена в файл `social_russia_data/data_regions_2020.csv`